**IMPORTS**


In [1]:
import datetime
import logging
import pandas as pd
from typing import Any, List, Literal, Tuple, TypeAlias, cast, Union
from matplotlib import pyplot as plt
from sklearn.multioutput import RegressorChain
from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import mean_squared_log_error
from xgboost import XGBRegressor
from gsp.model.utils import make_mw_in_groups, make_shift_in_groups, ColumnTransformerWrapper, show, get_nth_previous_working_date
from data import SCRAPED_STOCK_FILE_PATH, save_output

**FUNCTIONS**

In [2]:
from typing import Dict


def load_data() -> pd.DataFrame:
    return pd.read_csv(
        SCRAPED_STOCK_FILE_PATH,
        dtype={
            "Date": "period[D]",
            "Open": "float",
            "High": "float",
            "Low": "float",
            "Close": "float",
            "Volume": "int",
            "Area": "category",
            "Name": "category",
        },
    )


def clean_data(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy(deep=True)
    periods = pd.date_range(
        start=df["Date"].min().to_timestamp().date(), end=df["Date"].max().to_timestamp().date(), freq="B"
    )
    periods_df = pd.DataFrame({"Date": periods}, dtype="period[D]")

    clean_data = (
        cast(
            pd.DataFrame,
            periods_df.set_index("Date")
            .join(df.set_index("Date"))
            .set_index("Name", append=True)
            .sort_index()
            .unstack("Name")
            .ffill()
            .stack("Name", future_stack=True),  # type: ignore
        )
        .reset_index()
        .dropna(subset=["Name"])
        .set_index(["Date", "Name"])
    )

    return clean_data


def engineer_features(
    df: pd.DataFrame,
    categorical_features: List[str],
    shift_list: List[int],
    mwm_list: List[int],
) -> pd.DataFrame:

    df = df.copy()

    if "DayOfWeek" in categorical_features:
        df["DayOfWeek"] = cast(pd.PeriodIndex, df.index.get_level_values("Date")).dayofweek

    if "Month" in categorical_features:
        df["Month"] = cast(pd.PeriodIndex, df.index.get_level_values("Date")).month

    if "Year" in categorical_features:
        df["Year"] = cast(pd.PeriodIndex, df.index.get_level_values("Date")).year

    if "WeekOfYear" in categorical_features:
        df["WeekOfYear"] = cast(pd.PeriodIndex, df.index.get_level_values("Date")).week  # type: ignore

    if "DayOfMonth" in categorical_features:
        df["DayOfMonth"] = cast(pd.PeriodIndex, df.index.get_level_values("Date")).day

    if "Quarter" in categorical_features:
        df["Quarter"] = cast(pd.PeriodIndex, df.index.get_level_values("Date")).quarter

    if "AreaCat" in categorical_features:
        df["AreaCat"] = df["Area"]

    grouped_lags: List[pd.DataFrame | pd.Series] = [
        make_shift_in_groups(df, groupby=["Name"], column="Close", shift=shift_list),
        make_mw_in_groups(df, groupby=["Name"], column="Close", window=mwm_list),
    ]

    df_grouped_lags = df.join(grouped_lags, how="left")

    return df_grouped_lags


def process_data(
    df: pd.DataFrame,
    n_steps: int,
    categorical_features: List[str],
    starting_date_to_consider: datetime.date | None = None,
) -> Tuple[pd.DataFrame, pd.DataFrame]:

    df = df.copy()

    """First date where all stocks have data"""
    first_all_valid_date: datetime.date = (
        cast(pd.Period, df.unstack("Name").dropna().first_valid_index()).to_timestamp().date()
    )

    earliest_date: datetime.date = first_all_valid_date

    if starting_date_to_consider is not None and starting_date_to_consider < first_all_valid_date:
        show(
            f"WARNING: starting_date_to_consider ({starting_date_to_consider}) is before the first date where all stocks have data. Using {first_all_valid_date} instead."
        )
    elif starting_date_to_consider is not None:
        earliest_date = starting_date_to_consider

    show(f"Earliest date: {earliest_date.isoformat()}")

    ctw = ColumnTransformerWrapper(
        transformers=[
            ("onehot", OneHotEncoder(), categorical_features),
        ],
        remainder="passthrough",
    )

    X = ctw.fit_transform(df.drop(columns=["Adj Close", "Volume", "High", "Low", "Open", "Area"])).loc[
        earliest_date.isoformat() :
    ]
    y = make_shift_in_groups(df, groupby=["Name"], column="Close", shift=[-i for i in range(1, n_steps + 1)])

    y, X = y.align(X.dropna(), axis=0, join="inner")

    return X, y


def split_data(
    X: pd.DataFrame,
    y: pd.DataFrame,
    n_steps: int,
) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """A function that splits the time series data into train and test sets in regards
    to the number of forecasting steps. It will test the model on the last 2*n_steps.

    Args:
        X (pd.DataFrame): Features dataframe
        y (pd.DataFrame): Target dataframe
        n_steps (int): Number of steps of the forecasting task

    Returns:
        Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame]: X_train, y_train, X_test, y_test
    """
    latest_date = cast(pd.Period, X.index.get_level_values("Date").max()).to_timestamp().date()
    test_end_date = get_nth_previous_working_date(n=n_steps, date=latest_date)
    test_start_date = get_nth_previous_working_date(n=2 * n_steps - 1, date=latest_date)
    train_end_date = get_nth_previous_working_date(n=2 * n_steps, date=latest_date)

    X_train = X.loc[: train_end_date.isoformat()]
    y_train = y.loc[: train_end_date.isoformat()]
    X_test = X.loc[test_start_date.isoformat() : test_end_date.isoformat()]
    y_test = y.loc[test_start_date.isoformat() : test_end_date.isoformat()]

    return X_train, y_train, X_test, y_test


def create_model(**hyper_params):
    model = RegressorChain(XGBRegressor(**hyper_params))

    return model


def fit_and_predict(
    model: Union[RegressorChain, MultiOutputRegressor, LinearRegression],
    X_train: pd.DataFrame,
    y_train: pd.DataFrame,
    X_query: pd.DataFrame,
):
    model = model.fit(X_train, y_train)
    y_pred = pd.DataFrame(model.predict(X_query), index=X_query.index, columns=y_train.columns)  # type: ignore

    return y_pred


def convert_last_prediction_to_output(df: pd.DataFrame) -> pd.DataFrame:

    df = df.copy()

    latest_date = cast(pd.Period, df.index.get_level_values("Date").max()).to_timestamp().date()

    y_latest = cast(pd.DataFrame, df.loc[latest_date.isoformat()]).rename(
        columns={
            f"Close_lead_{i}": get_nth_previous_working_date(n=-i, date=latest_date).isoformat()
            for i in range(1, len(df.columns) + 1)
        }
    )

    y_output = (
        y_latest
        .reset_index()
        .melt(id_vars=["Name"], var_name="Date", value_name="Close")
        .astype({
            "Date": "period[D]",
            "Name": "category",
            "Close": "float",
        })
        .set_index(["Date", "Name"])
    )

    return y_output


def execute_test_run(
    n_steps: int, 
    categorical_features: List[str], 
    hyper_params: Dict = {}
) ->Tuple[pd.DataFrame, pd.DataFrame, float]:

    stocks = load_data()
    clean = clean_data(stocks)
    features = engineer_features(clean, categorical_features, shift_list=[1], mwm_list=[5])
    X, y = process_data(features, n_steps, categorical_features)
    X_train, y_train, X_test, y_test = split_data(X, y, n_steps)
    model = create_model(**hyper_params)
    y_test_pred = fit_and_predict(model, X_train, y_train, X_test)
    rmsle = cast(float, mean_squared_log_error(y_test, y_test_pred, squared=False))

    y_test_output = convert_last_prediction_to_output(y_test_pred)
    show(f"Root Mean Squared Log Error: {rmsle}")

    return features, y_test_output, rmsle


def execute_real_run(n_steps: int, categorical_features: List[str]) -> Tuple[pd.DataFrame, datetime.date]:

    stocks = load_data()
    clean = clean_data(stocks)
    features = engineer_features(clean, categorical_features, shift_list=[1], mwm_list=[5])
    X, y = process_data(features, n_steps, categorical_features)
    X_train, y_train = X.align(y.dropna(), axis=0, join="inner")

    prediction_date = cast(pd.Period, X.index.get_level_values("Date").max()).to_timestamp().date()
    X_query = X.loc[prediction_date.isoformat() :]

    model = create_model()
    y_pred = fit_and_predict(model, X_train, y_train, X_query)
    y_output = convert_last_prediction_to_output(y_pred)

    return y_output, prediction_date

**FORECASTING PROBLEM DEFINITION**

In [9]:
n_steps : int = 3

**TEST EXECUTION**

In [ ]:
features, y_test_output, rmsle = execute_test_run(n_steps=n_steps, categorical_features=["DayOfWeek"])

In [ ]:
diplay_last_n_days = 150
unique_names = y_test_output.index.get_level_values("Name").unique()
for name in unique_names:
    fig = plt.figure(figsize=(10, 5))
    fig.suptitle(name)
    
    cast(
        pd.DataFrame,
        features[features.index.get_level_values("Name") == name]
    ).reset_index("Name")["Close"].tail(diplay_last_n_days).plot(label=f"Close for {name}", ax = fig.gca())

    cast(
        pd.DataFrame,
        y_test_output[y_test_output.index.get_level_values("Name") == name]
    ).reset_index("Name")["Close"].plot(label=f"Predicted Close for {name}", ax = fig.gca())

    plt.show()

**REAL EXECUTION**

In [8]:
y_real_output, prediction_date = execute_real_run(n_steps=n_steps, categorical_features=["DayOfWeek"])
save_output(y_real_output, f"{prediction_date.isoformat()}_prediction_{n_steps}_steps.csv")

'Earliest date: 2004-08-20'